<ul class="breadcrumb">
  <li><a href="1_Conventional_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Conventional)</a></li>
  <li><a href="2_Linear_Additive_Genetic_Model.ipynb">Bayesian Linear Additive Genetic Model</a></li> 
  <li><a href="3_Genomic_Linear_Mixed_Model.ipynb">Bayesian Linear Mixed Models (Genomic Data)</a></li>
</ul>

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Bayesian Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 1: Load Packages</button> 

In [1]:
using JWAS,JWAS.Datasets,DataFrames,CSV

<button type="button" class="btn btn-lg btn-primary">Step 2: Read data</button> 

In [2]:
phenofile  = Datasets.dataset("example","phenotypes.txt")
pedfile    = Datasets.dataset("example","pedigree.txt")

phenotypes = CSV.read(phenofile,delim = ',',header=true,missingstrings=["NA"])
pedigree   = get_pedigree(pedfile,separator=",",header=true);

The delimiter in pedigree.txt is ','.
Finished!


In [3]:
first(phenotypes,5)

,ID,y1,y2,y3,x1,x2,x3,dam
,String,Float64,Float64⍰,Float64⍰,Float64,Int64,String,String
1,a1,-0.06,3.58,-1.18,0.9,2,m,0
2,a3,-2.07,3.19,missing,0.7,2,f,0
3,a4,-2.63,6.97,-0.83,0.6,1,m,a2
4,a5,2.31,missing,-1.52,0.4,2,m,a2
5,a6,0.93,4.87,-0.01,5.0,2,f,a3


<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Univariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [4]:
model_equation1  ="y1 = intercept + x1*x3 + x2 + x3 + ID + dam";

In [5]:
R      = 1.0
model1 = build_model(model_equation1,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [6]:
set_covariate(model1,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [7]:
G1 = 1.0
G2 = [1.0 0.5
      0.5 1.0]
set_random(model1,"x2",G1);
set_random(model1,"ID dam",pedigree,G2);

<button type="button" class="btn btn-lg btn-primary">Step 6:Run Analysis</button> 

In [8]:
outputMCMCsamples(model1,"x2")
out1=runMCMC(model1,phenotypes,chain_length=5000,output_samples_frequency=100);

A Linear Mixed Model was build using model equations:

y1 = intercept + x1*x3 + x2 + x3 + ID + dam

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1*x3           interaction  fixed                2
x2              factor       random               2
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
estimateScale                                 false
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint                                    false
missing_phenotypes                             true
update_priors_frequency

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:01


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [9]:
keys(out1)

Base.KeySet for a Dict{Any,Any} with 4 entries. Keys:
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "Posterior mean of residual variance"
  "Posterior mean of location parameters"

In [10]:
out1["Posterior mean of polygenic effects covariance matrix"]

2×2 Array{Float64,2}:
 5.05712  1.41212
 1.41212  2.35717

<div class="span5 alert alert-success">
 <font size="5" face="Georgia">Multivariate Linear Additive Genetic Model</font> 
</div>

<button type="button" class="btn btn-lg btn-primary">Step 3: Build Model Equations</button> 

In [11]:
model_equation2 ="y1 = intercept + x1 + x3 + ID + dam
                  y2 = intercept + x1 + x2 + x3 + ID
                  y3 = intercept + x1 + x1*x3 + x2 + ID";

In [12]:
R      = [1.0 0.5 0.5
          0.5 1.0 0.5
          0.5 0.5 1.0]
model2 = build_model(model_equation2,R);

<button type="button" class="btn btn-lg btn-primary">Step 4: Set Factors or Covariates</button> 

In [13]:
set_covariate(model2,"x1");

<button type="button" class="btn btn-lg btn-primary">Step 5: Set Random or Fixed Effects</button> 

In [14]:
G1 = [1.0 0.5
      0.5 1.0]
G2 = [1.0 0.5 0.5 0.5
      0.5 1.0 0.5 0.5
      0.5 0.5 1.0 0.5
      0.5 0.5 0.5 1.0]
set_random(model2,"x2",G1);
set_random(model2,"ID dam",pedigree,G2);

x2 is not found in model equation 1.
dam is not found in model equation 2.
dam is not found in model equation 3.


<button type="button" class="btn btn-lg btn-primary">Step 6: Run Analysis</button> 

In [15]:
outputMCMCsamples(model2,"x2")
out2=runMCMC(model2,phenotypes,chain_length=5000,output_samples_frequency=100,outputEBV=true);

A Linear Mixed Model was build using model equations:

y1 = intercept + x1 + x3 + ID + dam
y2 = intercept + x1 + x2 + x3 + ID
y3 = intercept + x1 + x1*x3 + x2 + ID

Model Information:

Term            C/F          F/R            nLevels
intercept       factor       fixed                1
x1              covariate    fixed                1
x3              factor       fixed                2
ID              factor       random              12
dam             factor       random              12
x2              factor       random               2
x1*x3           interaction  fixed                2

MCMC Information:

methods                        conventional (no markers)
chain_length                                   5000
burnin                                            0
estimateScale                                 false
starting_value                                false
printout_frequency                             5001
output_samples_frequency                        100
constraint

running MCMC for conventional (no markers)...100%|██████| Time: 0:00:02


<button type="button" class="btn btn-lg btn-primary">Check Results</button> 

In [16]:
keys(out2)

Base.KeySet for a Dict{Any,Any} with 6 entries. Keys:
  "EBV_y2"
  "Posterior mean of polygenic effects covariance matrix"
  "EBV_y1"
  "EBV_y3"
  "Posterior mean of residual variance"
  "Posterior mean of location parameters"

In [17]:
out2["Posterior mean of polygenic effects covariance matrix"]

4×4 Array{Float64,2}:
  3.5536    -2.87913   0.135462  0.321165
 -2.87913    4.90866   0.853486  0.211488
  0.135462   0.853486  1.12717   0.250172
  0.321165   0.211488  0.250172  1.39911 

In [18]:
out2["EBV_y3"]

,ID,Estimate
,Any,Any
1,a2,-0.118519
2,a1,0.0255316
3,a7,0.228192
4,a12,0.135108
5,a5,-0.302975
6,a3,0.351545
7,a4,0.163164
8,a6,0.126424
9,a10,0.0486478


In [19]:
out2["Posterior mean of location parameters"]

,Trait,Effect,Level,Estimate
,Any,Any,Any,Any
1,1,intercept,intercept,-15.3736
2,1,x1,x1,0.733355
3,1,x3,m,14.8988
4,1,x3,f,12.7948
5,1,ID,a2,0.162014
6,1,ID,a1,-0.262265
7,1,ID,a3,-0.164866
8,1,ID,a7,-0.102624
9,1,ID,a4,-1.85152
